# The battle of neighborhoods

By: Muhammad Hamza

## Intro

### The problem

With a population exceeding 150 Million, Lahore is undoubtedly one of the largest cities of Pakistan, the 5th largest country by population. However, Lahore has had a history of being unable to provide adequate medical care to its citizens, especially in the relatively underdeveloped areas. 

As the government has a low amount of resources, there is only a small number of hospitals that can be built to cover a number of areas. Some areas are fully covered while some areas are not covered at all. The number of hospitals to be built should cover areas with the lowest amount of healthcare facilities in their vicinity.

### Solution

In order to cater the current problem, I plan to create a map that would display the neighborhoods and the number of hospitals near them. These hospitals will be judged on the number of areas they cater, which will in turn identify the areas to which hospitals are very far. These identified points will then be used to analyze and determine the optimal points for building a new hospital.

## Data

### Collection Sources

Data will be collected from the following sources:
- [List of towns in Lahore](https://en.wikipedia.org/wiki/List_of_towns_in_Lahore)
- Geocode information from Geopy
- Hospital Data in Lahore from Foursquare

### Data Type

The data collected includes:
- list of neighborhoods in lahore
- list of hospitals in lahore with their neighborhoods
- Geo location data of hospitals and neighborhoods

## Methodology

- Different areas of Lahore will be individualized
- Number of hospitals in each area will be located
- Each hospital is assumed to have a coverage of 7km radius
- Identify which areas have no hospital coverage
- Determine which areas are optimal for a new hospital

## Code

In [13]:
### Import Libraries
import numpy as np #vectors used here
import time
import pandas as pd # For data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # Handle JSON files
import requests # Handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!pip install geopy 
from geopy.geocoders import OpenCage # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library
from folium import plugins

!pip install lxml
!pip install BeautifulSoup4

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [2]:
lahore_towns = 'https://en.wikipedia.org/wiki/List_of_towns_in_Lahore'
lahore = pd.read_html(lahore_towns)[0]
lahore

,#,Town,Union Council
0,1,Ravi,Begum Kot (UC 1) Kot Mohibbu (UC 2) Azizabad (...
1,2,Shalamar,Bhaghatpura (UC 15) Gujjarpura (UC 16) Rehmatp...
2,3,Wagha,Daroghawala (UC 42) Muhammad (UC 37) Sultan Me...
3,4,Aziz Bhatti,Harbanspura (UC 41) Rashidpura (UC 43) Fatehga...
4,5,Data Gunj Buksh,Kasurpura (UC 67) Ameenpura (UC 68) Kareem Bag...
5,6,Gulberg,Railway Colony (UC 31) Daras Barey Mian (UC 32...
6,7,Samanabad,Abu Bakar Siddique (UC 84) Shamnagar (UC 87) G...
7,8,Iqbal,Awan Town (UC 110: Hassan Town) Saidpur (UC 11...
8,9,Nishtar,Kamahan (UC 63) Hair (UC 64) Dhaloke (UC 66) B...
9,A,Cantonment*,"Lahore Cantonment (UC 152: Defence, Cavalry Gr..."


In [3]:
lahore_councils = pd.DataFrame(lahore['Union Council'])
lahore_councils

,Union Council
0,Begum Kot (UC 1) Kot Mohibbu (UC 2) Azizabad (...
1,Bhaghatpura (UC 15) Gujjarpura (UC 16) Rehmatp...
2,Daroghawala (UC 42) Muhammad (UC 37) Sultan Me...
3,Harbanspura (UC 41) Rashidpura (UC 43) Fatehga...
4,Kasurpura (UC 67) Ameenpura (UC 68) Kareem Bag...
5,Railway Colony (UC 31) Daras Barey Mian (UC 32...
6,Abu Bakar Siddique (UC 84) Shamnagar (UC 87) G...
7,Awan Town (UC 110: Hassan Town) Saidpur (UC 11...
8,Kamahan (UC 63) Hair (UC 64) Dhaloke (UC 66) B...
9,"Lahore Cantonment (UC 152: Defence, Cavalry Gr..."


In [4]:
lahore_towns = pd.DataFrame(lahore_councils['Union Council'].str.replace(r"\([^()]*\)",""))

In [5]:
lahore_towns = lahore_towns["Union Council"].str.split("  ", expand=True)

In [6]:
towns = []
df_shape = lahore_towns.shape
for i in range(0, df_shape[1]-1):
    for j in range(0, df_shape[0]-1):
        towns.append(lahore_towns[i][j])
towns=pd.DataFrame(towns, columns=['Neighborhood'])
towns=towns.dropna()
towns.reset_index(inplace=True, drop=True)

In [7]:
towns

,Neighborhood
0,Begum Kot
1,Bhaghatpura
2,Daroghawala
3,Harbanspura
4,Kasurpura
5,Railway Colony
6,Abu Bakar Siddique
7,Awan Town
8,Kamahan
9,Kot Mohibbu


In [8]:
geolocator = OpenCage(api_key="fc71ea1a55344ad38fb6a3477e2da517")
locationData = {"Latitude":[], "Longitude":[]}
for item in towns['Neighborhood']:
    location = geolocator.geocode(item + " Lahore, Punjab, PK")
    if(location is not None):
        print(item)
        locationData['Latitude'].append(location.latitude)
        locationData['Longitude'].append(location.longitude)

Begum Kot
Bhaghatpura
Daroghawala
Harbanspura
Kasurpura
Railway Colony
Abu Bakar Siddique
Awan Town
Kamahan
Kot Mohibbu
Gujjarpura
Muhammad
Rashidpura
Ameenpura
Daras Barey Mian
Shamnagar
Saidpur
Hair
Azizabad
Rehmatpura
Sultan Mehmood
Fatehgarh
Kareem Bagh
Bibi Pak Daman
Gulgasht
Sabzazar
Dhaloke
Faisal Bagh
Begampura
Dograi Kalan
Nabipura
Ganj Kalan
Garhi Shahu
Gulshan-e-Ravi
Canal
Bostan
Qaiser
Chah Miran
Muslimabad
Mughalpura
Bilalganj
Al Hamra
Babu Sabu
Bakar Mandi
Ismailnagar
Dhair
Bilal Bagh
Salamatpura
Mian Meer
Anarkali
Zaman Park
Rizwan Park
Johar Town
Sitara
Shahdara Bagh
Makhanpura
Lakhodher
Mustafabad
Gawalmandi
Gulberg
Sodiwal
Hanjarwal
Farid
Jia Musa
Kot Khawaja Saeed
Bhaseen
Ghaziabad
Sare Sultan
Makkah
Bahawalpur
Niaz Beg
Keer Kalan
Qila Lachhman Singh
Shad Bagh
Minhala
Tajbagh
Qila Gujar Singh
Naseerabad
Ichhra
Shahpur
Green Town
Palmandi
Wassanpura
Barki
Tajpura
Race Course Park
Garden Town
Naya Samanabad
Ali Raza Abad
Maryam
Siddiquepura
Faiz Bagh
Hadiara 
Al Faisal

In [9]:
locationData['Latitude'] = list(map(float, locationData['Latitude']))
locationData['Longitude'] = list(map(float, locationData['Longitude']))
locations = pd.DataFrame(locationData)

In [10]:
towns_new = pd.concat([towns, locations], axis=1)

In [21]:
towns_new

,Neighborhood,Latitude,Longitude
0,Begum Kot,31.613832,74.265741
1,Bhaghatpura,31.549720,74.343610
2,Daroghawala,31.587253,74.404541
3,Harbanspura,31.566945,74.396377
4,Kasurpura,31.549720,74.343610
5,Railway Colony,31.572491,74.351196
6,Abu Bakar Siddique,31.549720,74.343610
7,Awan Town,31.504234,74.263197
8,Kamahan,31.454575,74.381757
9,Kot Mohibbu,31.549720,74.343610


In [12]:
CLIENT_ID = '202FLYUI0SZUMOA40GAMW00WO4MB4TLD5PXZVJD122OXTAXM'
CLIENT_SECRET = 'VJWLFZVQB5DVRMVEHCBKABHXCJNKONB1FUKBPD25CY53IJTA'

Location(Lahore, Lahore District, Pakistan, (31.5656079, 74.3141775, 0.0))

In [25]:
m = folium.Map(location=[31.5656079, 74.3141775], zoom_start=15)
for i in range(0,len(towns_new)):
    folium.Marker([towns_new.iloc[i]['Longitude'], towns_new.iloc[i]['Latitude']], popup=towns_new.iloc[i]['Neighborhood']).add_to(m)
m